In [ ]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt
from data.diffusion import KnownLatentDiffusionDataset, KnownLatentDiffusionDatasetConfig
from models.encoder import DiffusionEncoder
from torch2jax import j2t, t2j
import lightning as L
from models.x_transformer import Encoder, ScaledSinusoidalEmbedding
from lightning_modules.metalearn import MetaLearningTask
import torch.nn as nn
from torch.utils.data import random_split, DataLoader
from einops import rearrange, repeat

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
/home/mila/l/leo.gagnon/late

In [15]:
class DeterministicEncoder(L.LightningModule):
    def __init__(
        self, pretrained_id: str, n_embd: int, batch_size: int, val_split: float, lr: float
    ):
        super().__init__()

        self.batch_size = batch_size
        self.val_split = val_split
        self.lr = lr
        self.n_embd = n_embd

        self.base_task = MetaLearningTask(pretrained_id)
        for param in self.base_task.parameters():
            param.requires_grad = False

        self.encoder = Encoder(
            dim=n_embd,
            depth=6,
            heads=8,
            attn_dropout=0.0,  # dropout post-attention
            ff_dropout=0.0,  # feedforward dropout
            rel_pos_bias=False,
            ff_glu=True,
            cross_attend=True
        )
        self.null_embedding = nn.Embedding(1, n_embd)
        #self.out_proj = nn.Linear(n_embd, self.base_task.model.encoder.cfg.n_embd)
        self.out_proj = nn.ModuleList([nn.Linear(n_embd, embedding.num_embeddings) for embedding in self.base_task.model.encoder.latent_embedding])
        self.seq_conditional_emb = nn.Embedding(
                num_embeddings=50,
                embedding_dim=n_embd,
            )
        self.seq_conditional_posemb = ScaledSinusoidalEmbedding(n_embd)
        self.norm = nn.LayerNorm(n_embd)

    def setup(self, stage):
        with torch.no_grad():
            self.train_data = KnownLatentDiffusionDataset(
                KnownLatentDiffusionDatasetConfig(context_length=[200,200]), self.base_task, None
            )

            #self.train_data, self.val_data = random_split(
            #    dataset, [1 - self.val_split, self.val_split]
            #)

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return opt

    def train_dataloader(self):
        return DataLoader(
            self.train_data,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=lambda x: x,
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_data,
            batch_size=self.batch_size,
            collate_fn=lambda x: x,
            shuffle=False,
        )

    def training_step(self, batch, batch_idx=None):
        latent, raw_latent = batch["latent"], batch["raw_latent"]
        cond_input_ids = batch["cond_input_ids"]

        cond = self.seq_conditional_emb(cond_input_ids)
        cond = cond + self.seq_conditional_posemb(cond)

        init_emb = repeat(self.null_embedding.weight, "1 d -> b 1 d", b=latent.shape[0])

        pred = self.encoder(
                init_emb,
                context=cond,
            )
        pred = self.norm(pred)
        pred = [proj(pred) for proj in self.out_proj]
        
        loss = sum([nn.functional.cross_entropy(pred[i].squeeze(), raw_latent[:,i]).mean() for i in range(len(pred))])
        #loss = torch.mean((pred - latent)**2)

        self.log(
            "train/loss",
            loss.detach().cpu().numpy().item(),
            prog_bar=True,
        )

        return loss

In [16]:
trainer = L.Trainer(
        max_steps=10000,
        accelerator='gpu',
        enable_checkpointing=False,
        val_check_interval=100,
        reload_dataloaders_every_n_epochs=1,
        check_val_every_n_epoch=None,
    )

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/mila/l/leo.gagnon/latent_control/venv/lib/pyth ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [18]:
deterministic_encoder = DeterministicEncoder("ekly943l", 512, 512, 0.1, 1e-4)

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt


In [ ]:
trainer.fit(deterministic_encoder)

/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                      | Params | Mode 
-----------------------------------------------------------------------------
0 | base_task              | MetaLearningTask          | 6.3 M  | train
1 | encoder                | Encoder                   | 31.5 M | train
2 | null_embedding         | Embedding                 | 512    | train
3 | out_proj               | ModuleList                | 13.9 K | train
4 | seq_conditional_emb    | Embedding                 | 25.6 K | train
5 | seq_conditional_posemb | ScaledSinusoidalEmbedding | 1      | train
6 | norm                   | LayerNorm                 | 1.0 K  | train
-----------------------------------------------------------------------------
31.5 M    Tr

Training: |          | 0/? [00:00<?, ?it/s]

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)
/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (24) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


In [8]:
deterministic_encoder.cuda();

In [9]:
batch = next(iter(deterministic_encoder.val_dataloader()))

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)


In [23]:
raw_latent, latent, cond_input_ids, _ = batch.values()

In [24]:
cond = deterministic_encoder.seq_conditional_emb(cond_input_ids)
cond = cond + deterministic_encoder.seq_conditional_posemb(cond)

init_emb = repeat(deterministic_encoder.null_embedding.weight, "1 d -> b 1 d", b=latent.shape[0])

pred = deterministic_encoder.encoder(
        init_emb,
        context=cond,
    )
pred = deterministic_encoder.norm(pred)
pred = deterministic_encoder.out_proj(pred)

In [32]:
((pred[0,0] - latent[0,0])**2).mean()

tensor(2.2939, device='cuda:0', grad_fn=<MeanBackward0>)

tensor([-6.0948e+00,  5.3547e+00, -4.7328e-01, -1.0948e+00, -7.2440e+00,
        -3.7185e+00, -8.8627e-01,  1.1151e+00,  2.3958e+00, -3.4279e-01,
        -7.1353e-02,  1.0859e+00,  2.0183e-02, -4.4475e+00, -2.3455e+00,
        -5.3475e-01, -3.5062e-01, -2.9890e+00, -1.5294e+00, -1.1318e+00,
        -8.6672e-01,  3.8297e+00, -2.5176e+00,  6.8801e-01,  5.6800e-01,
         1.1605e+00,  4.4664e+00, -7.4320e+00, -4.0966e+00,  4.2228e+00,
         9.6136e-01,  1.1208e-02, -2.0105e+00,  1.8194e+00,  2.0268e+00,
        -5.6192e-01, -3.4476e+00,  2.4000e+00, -1.7783e+00,  3.7460e+00,
        -1.4226e+00,  6.4360e-01,  6.6819e-01, -8.6129e-01,  1.9548e+00,
        -2.9551e+00,  6.7295e-01, -3.8257e+00, -2.9601e+00,  2.5221e+00,
        -1.0021e+00,  5.7923e-01,  6.0421e-01,  4.9703e+00,  3.6883e-01,
         1.7546e+00, -4.2353e+00,  6.5127e-01,  2.8964e+00,  1.3348e-01,
         3.7944e-01, -5.0825e+00, -4.5273e-01,  3.0447e+00,  1.9446e+00,
        -1.1596e+00,  3.1885e-01, -3.9824e+00,  5.4